# 2. Aplicar models

### CNN

In [1]:
import sys

from src import funcionshow
from src import librerias
import numpy as np
import time
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import KFold
from keras.utils import to_categorical

In [35]:
labels,imlabels,imnames,numim,numlab=librerias.crear_labels('../data/dataset/train/')
# Define the model architecture , (64, 240, 240, 3)shape dataset

model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape = (240,240,3)))
model.add(tf.keras.layers.MaxPool2D((2,2)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPool2D((2,2)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPool2D((2,2)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(6, activation='softmax'))

In [19]:
#Creating Batch size and Image shape
BATCH_SIZE=32
IMG_SHAPE=(224,224)
## Data Augmentation
train_datagen=ImageDataGenerator(rescale=1/255.0,zoom_range=0.2,shear_range=0.3,horizontal_flip=True,brightness_range=[0.5,1.5])
test_datagen=ImageDataGenerator(rescale=1/255.0)

In [38]:
#Defining Train, Test and Validation data
train_data=train_datagen.flow_from_directory("../data/dataset/test",target_size=IMG_SHAPE,batch_size=BATCH_SIZE,class_mode="categorical")

test_data=train_datagen.flow_from_directory("../data/dataset/train",target_size=IMG_SHAPE,batch_size=BATCH_SIZE,class_mode="categorical")
val_data=train_datagen.flow_from_directory("../data/dataset/test",target_size=IMG_SHAPE,batch_size=BATCH_SIZE,class_mode="categorical")

Found 2698 images belonging to 6 classes.
Found 10901 images belonging to 6 classes.
Found 2698 images belonging to 6 classes.


In [40]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics="accuracy")
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
85/85 [==============================] - 120s 1s/step - loss: 0.0975 - accuracy: 0.8858
Epoch 2/10
85/85 [==============================] - 104s 1s/step - loss: 0.0832 - accuracy: 0.9077
Epoch 3/10
85/85 [==============================] - 100s 1s/step - loss: 0.0959 - accuracy: 0.8933
Epoch 4/10
85/85 [==============================] - 103s 1s/step - loss: 0.0753 - accuracy: 0.9199
Epoch 5/10
85/85 [==============================] - 130s 2s/step - loss: 0.0643 - accuracy: 0.9322
Epoch 6/10
85/85 [==============================] - 100s 1s/step - loss: 0.0704 - accuracy: 0.9274
Epoch 7/10
85/85 [==============================] - 100s 1s/step - loss: 0.0581 - accuracy: 0.9366
Epoch 8/10
85/85 [==============================] - 117s 1s/step - loss: 0.0690 - accuracy: 0.9236
Epoch 9/10
85/85 [==============================] - 105s 1s/step - loss: 0.0691 - accuracy: 0.9211
Epoch 10/10
85/85 [==============================] - 100s 1s/step - loss: 0.0560 - accuracy: 0.9385


In [41]:
loss, acc = model.evaluate(test_data)
print('accuracy test: ',acc)
print('loss test: ',loss)

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_1/flatten_1/Reshape' defined at (most recent call last):
    File "C:\Users\moasi\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\moasi\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\moasi\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\moasi\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\moasi\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\moasi\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\moasi\anaconda3\lib\asyncio\base_events.py", line 601, in run_forever
      self._run_o